In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import math

#### vacab_size             代表总的token包含了多少词汇
#### d_model                代表每一个词的代表的向量维度
#### padding_idx            代表填充索引，当一个文本大小不够统一长度的时候，使用其填充
#### batch_size             代表一次前向/反向传播中处理样本的数量大小
#### seq_len                代表单个样本输入序列的token数量
#### max_len                代表模型能处理的最大序列长度（超过则截断）
#### drop_prob              Dropout概率，通过其随机丢弃部分神经元，增强模型泛化能力，防止过拟合

In [ ]:
from torch import Tensor

# 将输入的词汇表索引转换为指定维度的Embedding
class TokenEmbedding(nn.Embedding):
  def __init__(self, vocab_size, d_model):
    super(TokenEmbedding, self).__init__(vocab_size, d_model, padding_idx=1)

tensor([[0.4937, 0.2404, 0.8655, 0.1003],
        [0.1508, 0.9797, 0.8670, 0.4523],
        [0.7564, 0.9976, 0.6711, 0.5697],
        [0.6099, 0.3834, 0.0081, 0.9055]])


In [5]:
# 实现位置编码机制，解决自主意力机制的局限性
class PositionalEmbedding(nn.Module):
  def __init__(self, d_model, max_len, device):
    super(PositionalEmbedding, self).__init__()
    self.encoding=torch.zeros(max_len, d_model, device=device)
    
    # 位置编码不需要训练
    self.encoding.requires_grad = False
    pos = torch.arange(0, max_len, device=device)
    pos = pos.float().unsqueeze(dim=1)
    _2i = torch.arange(0, d_model, step=2, device=device).float()
    self.encoding[:, 0:: 2] = torch.sin(pos/(10000**(_/d_model)))
    self.encoding[:, 0:: 2] = torch.cos(pos/(10000**(_/d_model)))
  
  def forward(self, x):
    batch_size, seq_len = x.size()
    return self.encoding[: seq_len,: ]

In [ ]:
# 将token 嵌入和位置编码结合，并对应用dropout正则化
class TransformerEmbedding(nn.Module):
  def __init__(self, vocab_size, d_model, max_len, drop_prob, device):
    super(TransformerEmbedding, self).__init__()
    self.tok_emb = TokenEmbedding(vocab_size, d_model)
    self.pos_emb = PositionalEmbedding(d_model, max_len, device)
    self.drop_out = nn.Dropout(p=drop_prob)

  def forward(self, x):
    tok_emb = self.tok_emb(x)
    pos_emb = self.pos_emb(x)
    return self.drop_out(tok_emb+pos_emb)